# Inteligencia Artificial - Laboratorio 9 -
### Integrantes
- Mark Albrand/21004
- Jimena Hernandez/21199

# Tasks 1 - Teoría

1. Diga cual es la diferencia entre Modelos de Markov y Hidden Markov Models

La principal diferencia entre un Modelo de Markov y un Modelo de Markov Oculto (HMM) es que en el HMM no hay una correspondencia directa entre las observaciones y los estados del modelo. En un HMM, los estados son ocultos y solo las observaciones son visibles (Purdue University, s.f.).
Esto hace que no sea posible determinar de manera directa en qué estado estaba la máquina cuando produjo una determinada observación. En cambio, en un modelo de Markov, los estados son visibles y se puede determinar directamente en qué estado estaba la máquina en un momento dado.

*Purdue University. (s. f.). Markov chains and hidden Markov models. https://www.stat.purdue.edu/~junxie/topic3.pdf*

2. Investigue qué son los factorial HMM (Hidden Markov Models)
Estos modelos combinan la estructura de transición de estados de los HMM con las representaciones distribuidas de los CVQ. Esto puede representarse de mejor forma en la siguiente figura:
<div style="width:400px">

![ Hidden Markov model & Factorial hidden Markov model](mkd.png)

</div>

Cada modelo de Markov subyacente, tiene un estado específico y su propia matriz que determina las probabilidades de transición entre estados. Asimismo en los CVQ cada estado dentro de un modelo es exclusivo.

Los estados en estos modelos son representados por vectores que principalmente tienen 0's excepto por un solo uno, indicando el estado activo.
*Ghahramani, Z., & Jordan, M. I. (1996). Factorial hidden Markov models (A.I. Memo No. 1561; C.B.C.L. Paper No. 130). Massachusetts Institute of Technology, Artificial Intelligence Laboratory and Center for Biological and Computational Learning, Department of Brain and Cognitive Sciences.*

3. Especifique en sus propias palabras el algoritmo Forward Backward para HMM

El algoritmo forward-backward se usa para calcular la probabilidad de que un HMM genere una secuencia de observaciones dadas. El algoritmo se divide en dos pasos: el paso forward y el paso backward.

    En el paso forward se calcula la probabilidad de ver las observaciones hasta un momento específico, dado que el modelo está en un estado particular en un momento dado.
    En el paso backward se calcula la probabilidad de ver las observaciones restantes, dado que el modelo está en un estado particular en un momento dado. 

Con estos dos pasos es posible calcular la probabilidad de que el modelo genere una secuencia de observaciones dada.

*Jurafsky, D., & Martin, J. H. (2024). Speech and Language Processing. Recuperado de https://web.stanford.edu/~jurafsky/slp3/A.pdf*

4. En el algoritmo de Forward Backward, por qué es necesario el paso de Backward (puede escribir ejemplos
o casos para responder esta pregunta.

Sabiendo que forward consiste en una suma de pesos de caminos desde “start” a “H i =h i ” y backward consiste en la suma de pesos de caminos desde “H i =h i ” a “end. Backward es de suma importancia para poder tener una probabilidad mas exacta, analizando los estados siguientes y no únicamente los anteriores. Con el paso Backward podemos saber como un estado lleva a observaciones futuras, haciendo que la probabilidad sea más precisa como se mencionó anteriormente y aportando un contexto futuro a cada estado. *(Suriano.A, 2024)*

# Task 2 - Algoritmo Forward Backward en HMM

In [6]:
import random


class HMM:
    def __init__(self, states: list[str], observations: list[str], initial_prob: dict[str, int], transition_prob: dict[str, dict[str, int]], emission_prob: dict[str, dict[str, int]]):
        self.states: list[str] = states  # Lista de estados posibles del HMM
        self.observations: list[str] = observations  # Lista de observaciones posibles del HMM
        self.initial_prob: dict[str, int] = initial_prob  # Probabilidad inicial de cada estado
        self.transition_prob: dict[str, dict[str, int]] = transition_prob  # Probabilidad de transición entre estados. Se refiere a la probabilidad de pasar de un estado a otro
        self.emission_prob: dict[str, dict[str, int]] = emission_prob  # Probabilidad de emitir una observación dado un estado. Se refiere a la probabilidad de observar una secuencia de observaciones dado un estado

    def generate_sequence(self, length: int):
        """ Generar una secuencia de observaciones basadas en el HMM"""
        sequence = []
        state = random.choices(self.states, weights=[self.initial_prob[s] for s in self.states])[0]
        for _ in range(length):
            # Seleccionar una observación basada en la probabilidad de emisión del estado actual
            observation = random.choices(self.observations, weights=[self.emission_prob[state][o] for o in self.observations])[0]
            sequence.append((state, observation))
            state = random.choices(self.states, weights=[self.transition_prob[state][s] for s in self.states])[0]
        return sequence
    
    def forward(self, sequence):
        """ Implementar el paso forward del algoritmo forward-backward"""
        forward_prob = []
        for i, (state, observation) in enumerate(sequence):
            if i == 0:  # Incialización con las probabilidades del estado inicial multiplicadas por las probabilidades de emisión.
                forward_prob.append({s: self.initial_prob[s] * self.emission_prob[s][observation] for s in self.states})
            else:  # Se calcula la probabilidad de cada estado basandose en la probabilidad de los estados anteriores
                forward_prob.append({s: sum(forward_prob[i - 1][s1] * self.transition_prob[s1][s] * self.emission_prob[s][observation] for s1 in self.states) for s in self.states})
        return forward_prob
    
    def backward(self, sequence):
        """ Implementar el paso backward del algoritmo forward-backward"""
        backward_prob = [{} for _ in sequence]  # Lista de diccionarios para almacenar las probabilidades backward
        for i in range(len(sequence) - 1, -1, -1):
            state, observation = sequence[i]
            if i == len(sequence) - 1:  # Inicialización con las probabilidades del estado final multiplicadas por las probabilidades de emisión
                backward_prob[i] = {s: 1 for s in self.states}  # Inicialización con 1
            else:  # Se calcula la probabilidad backward de cada estado basandose en la probabilidad de los estados siguientes 
                backward_prob[i] = {s: sum(self.transition_prob[s][s1] * self.emission_prob[s1][sequence[i + 1][1]] * backward_prob[i + 1][s1] for s1 in self.states) for s in self.states}  # Cálculo de la probabilidad backward 
        
        return backward_prob
    
    def compute_state_probability(self, sequence):
        """ Combine probabilidades forward y backward para obtener la probabilidad de un estado """
        forward_prob = self.forward(sequence)
        backward_prob = self.backward(sequence)
        # Combinación de probabilidades forward y backward para obtener la probabilidad de un estadoi
        state_prob = [{s: forward_prob[i][s] * backward_prob[i][s] for s in self.states} for i in range(len(sequence))]
        return state_prob

In [7]:
# Uso y datos
states = ['Sunny', 'Rainy']
observations = ['Sunny', 'Sunny', 'Rainy'] 
initial_prob = {'Sunny': 0.5, 'Rainy': 0.5}
transition_prob = {'Sunny': {'Sunny': 0.8, 'Rainy': 0.2}, 'Rainy': {'Sunny': 0.4, 'Rainy': 0.6}}
emission_prob = {'Sunny': {'Sunny': 0.8, 'Rainy': 0.2}, 'Rainy': {'Sunny': 0.3, 'Rainy': 0.7}}
hmm = HMM(states, observations, initial_prob, transition_prob, emission_prob)

In [9]:
# Generar una secuencia de observaciones
obs_seq = hmm.generate_sequence(5)
print("Secuencia de observaciones generada:", obs_seq)

#Calculo de probabilidades forward
forward_prob = hmm.forward(obs_seq)
print("Probabilidades forward:", forward_prob)

#Calculo de probabilidades backward
backward_prob = hmm.backward(obs_seq)
print("Probabilidades backward:", backward_prob)

#Calculo de probabilidades de estados
state_prob = hmm.compute_state_probability(obs_seq)
print("Probabilidades de estados:", state_prob)


Secuencia de observaciones generada: [('Sunny', 'Sunny'), ('Sunny', 'Sunny'), ('Rainy', 'Sunny'), ('Rainy', 'Sunny'), ('Rainy', 'Rainy')]
Probabilidades forward: [{'Sunny': 0.4, 'Rainy': 0.15}, {'Sunny': 0.30400000000000005, 'Rainy': 0.051000000000000004}, {'Sunny': 0.21088000000000007, 'Rainy': 0.027420000000000003}, {'Sunny': 0.14373760000000008, 'Rainy': 0.017588400000000004}, {'Sunny': 0.024405088000000012, 'Rainy': 0.02751039200000001}]
Probabilidades backward: [{'Sunny': 0.10434480000000007, 'Rainy': 0.06785040000000003}, {'Sunny': 0.15324000000000007, 'Rainy': 0.10452000000000003}, {'Sunny': 0.22200000000000006, 'Rainy': 0.18600000000000003}, {'Sunny': 0.30000000000000004, 'Rainy': 0.5}, {'Sunny': 1, 'Rainy': 1}]
Probabilidades de estados: [{'Sunny': 0.04173792000000003, 'Rainy': 0.010177560000000004}, {'Sunny': 0.04658496000000003, 'Rainy': 0.005330520000000002}, {'Sunny': 0.04681536000000003, 'Rainy': 0.005100120000000001}, {'Sunny': 0.04312128000000003, 'Rainy': 0.00879420000